In [24]:
import torch, numpy as np, config, os
from PIL import Image, ImageDraw
from loss import _decode_boxes         # already defined in Loss textdoc
from loss import _decode_predictions   # same – decodes + score-threshold

CELL = 896 // config.S                # = 8 px for S = 112

# ----------------------------------------------------------------------
# helpers ---------------------------------------------------------------
# ----------------------------------------------------------------------
# ----------------------------------------------------------
def _rel2abs(cx, cy, w, h):
    cx, cy, w, h = cx*896, cy*896, w*896, h*896
    return [cx - w/2, cy - h/2, cx + w/2, cy + h/2]

def _gt_box_from_row(row):
    cx = (row['cx'] + row['tx']) / config.S
    cy = (row['cy'] + row['ty']) / config.S
    w  = np.exp(row['tw']) * config.ANCHORS[0][0]
    h  = np.exp(row['th']) * config.ANCHORS[0][1]
    return _rel2abs(cx, cy, w, h)

# ----------------------------------------------------------
def render_prediction(model, gt_df, img_id, score_thresh=0.3, device='cuda'):
    """Save & return a 896×896 image with GT (green) and preds (red)."""
    ann = gt_df[gt_df.img_id == img_id]
    if ann.empty:
        raise ValueError(f"no rows with img_id={img_id}")

    img_path = os.path.join(config.img_dir, ann['filename'].iloc[0])
    pil = Image.open(img_path).convert('RGB').resize((896, 896))

    # forward pass -----------------------------------------------------
    model = model.to(device).eval()
    x = (torch.tensor(np.asarray(pil), dtype=torch.float32)
            .permute(2,0,1).unsqueeze(0).div_(255).to(device))
    with torch.no_grad():
        preds = model(x).cpu()
    det = _decode_predictions(preds, score_thresh)[0]

    # draw -------------------------------------------------------------
    draw = ImageDraw.Draw(pil, 'RGBA')

    # GT green
    for _, row in ann.iterrows():
        draw.rectangle(_gt_box_from_row(row), outline=(0,255,0,200), width=2)

    #predictions red
    for tx, ty, w, h in det['boxes']:
        draw.rectangle(_rel2abs(tx, ty, w, h),
                       outline=(255,0,0,180), width=2)

        # put text at top-left of the box
        # x_text, y_text = xy[0] + 2, xy[1] + 2
        # score = det['scores'][k].item()
        # draw.text((x_text, y_text),
        #         f"{score:.2f}",          # or f"{k}" for box index
        #         fill=(255,0,0,255),
        #         font=font)

    out_path = os.path.join(os.path.dirname(os.path.dirname(img_path)), "evaluation", f"{img_id}_viz.png")
    pil.save(out_path)
    print(f"[render_prediction] saved → {out_path}")
    return pil


## Predictions

In [ ]:
import util
import model as m

de = util.DataExtractor()
anns = de.normalizedData()
img_ids = anns["img_id"].unique().tolist()


Saving to: c:\Users\alexh\Desktop\cv2\obb_anns_hausarbeit\ds2_dense\ds2_dense\gt_space.json


In [ ]:
model = m.YOLOv2Heavy()
model = util.loadModel("concord050", model, device='cuda')
_ = render_prediction(model, anns, img_id=img_ids[0], score_thresh=0.5)


c:\Users\alexh\Desktop\cv2\obb_anns_hausarbeit\util.py:148: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path, map_location=device))


[loadMode] Loaded weights from model_dumps\concorde050.pth
[render_prediction] saved → c:\Users\alexh\Desktop\cv2\obb_anns_hausarbeit\ds2_dense\ds2_dense\evaluation\679_viz.png
